# Limpieza de Disney+

## Pedir datos a la API

In [2]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq

import sys, os
sys.path.append(os.path.abspath(".."))

from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [3]:
dfdisney_raw = buscar_peliculas('Disney+')

<Response [200]> 4 800


Fecha de consulta en la API: 20-10-2025 22:00

Verificamos que no haya perdida de datos:

In [5]:
largo_dataframe = len(dfdisney_raw[0])
largo_segun_API = dfdisney_raw[1]
print([largo_dataframe, largo_segun_API])

[800, 800]


Los largos coinciden

In [6]:
dfdisney1 = dfdisney_raw[0]

In [7]:
dfdisney1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1874486,Predator: Killer of Killers,2025,tt36463894,1376434,movie,movie
1,1685606,Thunderbolts*,2025,tt20969586,986056,movie,movie
2,1508465,Snow White,2025,tt6208148,447273,movie,movie
3,1560295,Doctor Strange in the Multiverse of Madness,2022,tt9419884,453395,movie,movie
4,138018,Avatar: The Way of Water,2022,tt1630029,76600,movie,movie
...,...,...,...,...,...,...,...
795,1789373,Fly,2024,tt31189863,1242392,movie,movie
796,1942789,Jade Eyed Leopard,2020,tt32059187,790058,movie,movie
797,1829508,Protecting Paradise: The Story of Niue,2024,tt32421420,1300132,movie,movie
798,1819120,John Williams in Tokyo,2024,tt36131268,1284678,movie,movie


Guardamos estos datos crudos en un csv

In [8]:
dfdisney1.to_csv('../data/raw_api_disney.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la película en la API
- `title`: Nombre de la película
- `year`: Año de estreno de la película
- `imdb_id`: Identificador de la película en IMDB
- `tmdb_id`: Identificador de la película en TMDB
- `tmdb_type`: Tipo del título en TMDB
- `type`: Tipo del título en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo películas, la columna `type` y `tmdb_type` no deberían entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [9]:
dfdisney1['type'].unique()

array(['movie'], dtype=object)

In [10]:
dfdisney1['tmdb_type'].unique()

array(['movie'], dtype=object)

Al ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [11]:
dfdisney2 = dfdisney1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores Na

In [12]:
resultados = {}
for i in dfdisney2.columns:
    resultados[i] = dfdisney2[i].isna().sum()
resultados

{'id': np.int64(0),
 'title': np.int64(0),
 'year': np.int64(0),
 'imdb_id': np.int64(0),
 'tmdb_id': np.int64(0)}

### Duplicados

In [13]:
dfdisney2.duplicated().sum()

np.int64(0)

#### id
Deberia ser unico para cada pelicula

In [14]:
dfdisney2.duplicated('id').sum()

np.int64(0)

#### title
Es posible que se repita, ya que hay películas con el mismo título

In [15]:
dfdisney2.duplicated('title').sum()

np.int64(24)

revisemos porque se repiten los titulos de las peliculas

In [18]:
dfdisney2[dfdisney2['title'] == 'Snow White']

,id,title,year,imdb_id,tmdb_id
2,1508465,Snow White,2025,tt6208148,447273
622,1971665,Snow White,2025,tt35983585,1449951
710,1883601,Snow White,2025,tt32370897,1387127


Son distintas películas con mismo nombre

#### imdb_id

In [19]:
dfdisney2.duplicated('imdb_id').sum()

np.int64(0)

### Tipos de columnas

In [20]:
dfdisney2.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

Los tipos de las columnas están bien

### Analisis de columna `year`

In [21]:
dfdisney2['year'].describe()

count     800.000000
mean     2005.835000
std        17.800951
min      1938.000000
25%      1998.000000
50%      2010.000000
75%      2019.000000
max      2025.000000
Name: year, dtype: float64

todos están dentro de un rango lógico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [22]:
dfdisney2.to_csv('../data/api_disney.csv', index=False)